<a href="https://colab.research.google.com/github/decouflet/7506R-2C2022-GRUPO15/blob/main/TP1_orga_del_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TP1

In [ ]:
#Manipulacion de datos y funciones matemáticas
import pandas as pd
import numpy as np

#Visualizaciones
import seaborn as sns
import matplotlib.pyplot as plt

#Modelo Lineal
from sklearn.linear_model import LinearRegression

#Metricas para evaluar modelos
from sklearn import metrics

In [6]:
import pandas as pd

properati = pd.read_csv('./properati_argentina_2021.csv')

In [10]:
properati.shape

(460154, 20)

Hay 460154 filas y 20 columnas

In [11]:
properati.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460154 entries, 0 to 460153
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id                        460154 non-null  object 
 1   start_date                460154 non-null  object 
 2   end_date                  460154 non-null  object 
 3   created_on                460154 non-null  object 
 4   latitud                   419740 non-null  float64
 5   longitud                  419740 non-null  float64
 6   place_l2                  460154 non-null  object 
 7   place_l3                  437665 non-null  object 
 8   place_l4                  139020 non-null  object 
 9   place_l5                  2430 non-null    object 
 10  place_l6                  0 non-null       float64
 11  operation                 460154 non-null  object 
 12  property_type             460154 non-null  object 
 13  property_rooms            368498 non-null  f

Mostramos un poco de informacion en general del dataset

In [12]:
properati.dtypes

id                           object
start_date                   object
end_date                     object
created_on                   object
latitud                     float64
longitud                    float64
place_l2                     object
place_l3                     object
place_l4                     object
place_l5                     object
place_l6                    float64
operation                    object
property_type                object
property_rooms              float64
property_bedrooms           float64
property_surface_total      float64
property_surface_covered    float64
property_price              float64
property_currency            object
property_title               object
dtype: object

Estos son los tipos de las 20 columnas

In [14]:
properati.isnull().sum()

id                               0
start_date                       0
end_date                         0
created_on                       0
latitud                      40414
longitud                     40414
place_l2                         0
place_l3                     22489
place_l4                    321134
place_l5                    457724
place_l6                    460154
operation                        0
property_type                    0
property_rooms               91656
property_bedrooms           116041
property_surface_total       62341
property_surface_covered     32238
property_price               18001
property_currency            18564
property_title                   0
dtype: int64

Vemos que hay filas con datos faltantes

In [43]:

properati_casa_depto = properati[properati.property_type.isin(['Casa', 'PH','Departamento'])]
properati_zona = properati_casa_depto[properati_casa_depto.place_l2.isin(['Capital Federal'])]
properati_operacion = properati_zona[properati_zona.operation.isin(['Venta'])]
properati_limpio = properati_operacion[properati_operacion.property_currency.isin(['USD'])]
properati_limpio.shape

(94249, 20)

Realizamos filtrado por:
    -Tipo de propiedad
    -Zona, que corresponda a capital
    -Tipo de Operacion
    -Tipo de moneda
Tomamos en cuenta que es excluyente que los datos no tengan datos faltantes para realizar este primer filtro
Vemos que el dataframe reduce su tamaño

In [40]:
#CON ESTE COMANDO SE GUARDA
#properati_limpio2 = pd.DataFrame(properati_limpio)
#properati_limpio2.to_csv('properatilimpio.csv',index = False)

In [41]:
properati_limpio.dtypes

id                           object
start_date                   object
end_date                     object
created_on                   object
latitud                     float64
longitud                    float64
place_l2                     object
place_l3                     object
place_l4                     object
place_l5                     object
place_l6                    float64
operation                    object
property_type                object
property_rooms              float64
property_bedrooms           float64
property_surface_total      float64
property_surface_covered    float64
property_price              float64
property_currency            object
property_title               object
dtype: object

Vemos que las columas *start_date, end_date y created_on* son tipo OBJECT, lo cual decidimos pasarlo a su tipo de dato correspondiente el cual es *datetime*

In [52]:
#properati_limpio['start_date'].unique().size
#properati2 = pd.to_datetime(properati_limpio['start_date'])
#properati2.unique()

In [58]:
from sklearn.model_selection import train_test_split

train_set, train_test = train_test_split(properati_limpio, test_size = 0.2, random_state = 1)


,id,start_date,end_date,created_on,latitud,longitud,place_l2,place_l3,place_l4,place_l5,place_l6,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,property_title
230935,MkAjgnqY718846QN3NHGGg==,2021-07-27,2021-10-06,2021-07-27,-34.554682,-58.478302,Capital Federal,Saavedra,NaN,NaN,NaN,Venta,Departamento,3.0,2.0,127.0,78.0,219000.0,USD,VENTA SAAVEDRA DUPLEX 3 AMBIENTES CON TERRAZA
111093,kLfpkVkZJFavNtzOa/iuhw==,2021-10-19,2022-01-27,2021-10-19,-34.610723,-58.394176,Capital Federal,Congreso,NaN,NaN,NaN,Venta,Departamento,3.0,2.0,NaN,62.0,97500.0,USD,Gran 2 ambientes a solo 2 cuadras del Congreso...
376991,fvsSd7SlsWoCXFDyZTW3YA==,2021-03-18,2021-06-05,2021-03-18,-34.610046,-58.361382,Capital Federal,Puerto Madero,NaN,NaN,NaN,Venta,Departamento,3.0,2.0,171.0,171.0,1000000.0,USD,Departamento - Puerto Madero
267784,DvMe86waQuCnR7ogK9gCPQ==,2021-07-09,2021-07-20,2021-07-09,-34.629075,-58.367129,Capital Federal,Boca,NaN,NaN,NaN,Venta,Departamento,2.0,1.0,41.0,38.0,60000.0,USD,VENTA IMPECABLE DEPARTAMENTO 2 AMBIENTES CON B...
437997,zTJvxpkcEhF0LbBgTG0a8Q==,2021-10-27,2021-11-03,2021-10-27,-34.575360,-58.441559,Capital Federal,Colegiales,NaN,NaN,NaN,Venta,Departamento,2.0,1.0,47.0,42.0,115150.0,USD,VENTA POZO 2 AMBIENTES C/BALCÓN (UF.504) COLEG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135561,I2M3P7ziKULrgdRQQdG1zA==,2021-04-06,2021-04-06,2021-04-06,-34.613398,-58.359734,Capital Federal,Puerto Madero,NaN,NaN,NaN,Venta,Departamento,4.0,3.0,177.0,160.0,1090000.0,USD,Departamento - Puerto Madero
267906,+RU+k7s1E9VeBWJb853YFw==,2021-07-09,2022-03-31,2021-07-09,-34.604427,-58.418079,Capital Federal,Almagro,NaN,NaN,NaN,Venta,Departamento,1.0,NaN,32.0,30.0,68500.0,USD,Bajó de Precio! Vende ya!
252747,tzuJfgL4wUlu9Sz1iVN/aA==,2021-06-18,2021-06-19,2021-06-18,-34.638328,-58.473294,Capital Federal,Floresta,NaN,NaN,NaN,Venta,Departamento,1.0,NaN,41.0,27.0,58900.0,USD,VENTA MONOAMBIENTE FLORESTA CON PATIO A ESTRENAR
426120,0ksKHOfNFYWoAdR/EgIUAA==,2021-12-16,2022-01-27,2021-12-16,-34.592111,-58.432919,Capital Federal,Palermo,NaN,NaN,NaN,Venta,Departamento,1.0,1.0,40.0,37.0,81000.0,USD,Departamento - Palermo
